## GStreamer DeckLink

- https://gstreamer.freedesktop.org/documentation/installing/on-linux.html?gi-language=python


conda activate GST-DeckLink

sudo apt-get install libgstreamer1.0-dev libgstreamer-plugins-base1.0-dev libgstreamer-plugins-bad1.0-dev gstreamer1.0-plugins-base gstreamer1.0-plugins-good gstreamer1.0-plugins-bad gstreamer1.0-plugins-ugly gstreamer1.0-libav gstreamer1.0-tools gstreamer1.0-x gstreamer1.0-alsa gstreamer1.0-gl gstreamer1.0-gtk3 gstreamer1.0-qt5 gstreamer1.0-pulseaudio

sudo apt install python3-gi gir1.2-gstreamer-1.0

conda install -c conda-forge gstreamer gst-plugins-base gst-plugins-good gst-plugins-bad gst-plugins-ugly gst-libav gst-python pygobject

- https://github.com/GStreamer/gstreamer/tree/main/subprojects/gst-docs/examples 

**Repositories:**
- https://gitlab.freedesktop.org/gstreamer/gstreamer
- https://github.com/gkralik/python-gst-tutorial
- https://github.com/Julusian/decklink-rs
- https://gist.github.com/hum4n0id/cda96fb07a34300cdb2c0e314c14df0a

**Theory:**
- https://gstreamer.freedesktop.org/documentation/?gi-language=python
- https://brettviren.github.io/pygst-tutorial-org/pygst-tutorial.html

In [11]:
import sys
import gi

gi.require_version('GLib', '2.0')
gi.require_version('GObject', '2.0')
gi.require_version('Gst', '1.0')

from gi.repository import Gst, GObject, GLib

pipeline = None
bus = None
message = None

# initialize GStreamer
Gst.init(sys.argv[1:])

# build the pipeline
pipeline = Gst.parse_launch(
    "playbin uri=https://gstreamer.freedesktop.org/data/media/sintel_trailer-480p.webm"
)

# start playing
pipeline.set_state(Gst.State.PLAYING)

# wait until EOS or error
bus = pipeline.get_bus()
msg = bus.timed_pop_filtered(
    Gst.CLOCK_TIME_NONE,
    Gst.MessageType.ERROR | Gst.MessageType.EOS
)

# free resources
pipeline.set_state(Gst.State.NULL)

<enum GST_STATE_CHANGE_SUCCESS of type Gst.StateChangeReturn>

**DeckLink Duo Info:**
- **device-number** = 0 (0x00000000)
- **model-name** = "DeckLink\ Duo\ 2"
- **display-name** = "DeckLink\ Duo\ \(1\)"
- **max-channels** = 16 (0x00000010)
- **supports-format-detection** = true | gst-launch-1.0 decklinkvideosrc
- **Video Source**: `decklinkvideosrc`
- **Video Mode**: `1080p59.94` (1920x1080, progressive, 59.94 fps)
- **Pixel Format**: `8-bit YUV 4:2:2` (UYVY)
- **Audio Source**: `decklinkaudiosrc`
- **Output**: Video frames are converted to `BGR` format for easy use with libraries like OpenCV and NumPy. Audio is captured as raw samples.

Run this commmand: 
- gst-inspect-1.0 decklink
- gst-device-monitor-1.0 Video/Source

In [12]:
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst, GLib

Gst.init(None)
print("GStreamer initialized.")

GStreamer initialized.


In [16]:
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst, GObject

def list_devices(device_class):
    """
    Lists available GStreamer devices for a given device class.
    
    :param device_class: The GStreamer device class string, e.g., "Video/Source" or "Audio/Source".
    """
    monitor = Gst.DeviceMonitor.new()
    monitor.add_filter(device_class, None)
    monitor.start()

    devices = monitor.get_devices()
    print(f"--- Available Devices for '{device_class}' ({len(devices)}) ---")
    if not devices:
        print("No devices found.")
    for i, device in enumerate(devices):
        props = device.get_properties()
        # Display name provides a user-friendly name
        display_name = device.get_display_name()
        # The 'device.path' or 'device' property in the capabilities often contains the system path (e.g., /dev/video0)
        device_path = props.get_string("device.path") or props.get_string("device")
        print(f"  {i+1}. Name: {display_name}, Path: {device_path}")

    monitor.stop()

def main():
    # Initialize GStreamer
    Gst.init(None)
    
    # List video input devices
    list_devices("Video/Source")
    
    print("\n" + "="*40 + "\n")
    
    # List audio input devices
    list_devices("Audio/Source")

if __name__ == "__main__":
    main()


--- Available Devices for 'Video/Source' (0) ---
No devices found.


--- Available Devices for 'Audio/Source' (0) ---
No devices found.


### Blackmagic DeckLink Duo - Video/Audio Specs

In [13]:
# List All DeckLink Devices

def list_decklink_devices():
    device_monitor = Gst.DeviceMonitor()
    device_monitor.add_filter("Video/Source", None)
    device_monitor.start()
    devices = device_monitor.get_devices()
    decklink_devices = []
    for dev in devices:
        if 'decklink' in dev.get_display_name().lower():
            decklink_devices.append(dev)
    device_monitor.stop()
    return decklink_devices

decklink_devices = list_decklink_devices()

if decklink_devices:
    for idx, dev in enumerate(decklink_devices):
        print(f"Device {idx}: {dev.get_display_name()}")
else:
    print("No DeckLink devices found.")


No DeckLink devices found.


In [14]:
# Video Modes and Pixel Formats

def print_device_caps(device):
    print(f"\nCapabilities for device: {device.get_display_name()}")
    vid_caps = device.get_caps()
    for i in range(vid_caps.get_size()):
        structure = vid_caps.get_structure(i)
        print(f"  {structure.to_string()}")

if decklink_devices:
    print_device_caps(decklink_devices[0])
else:
    print("No DeckLink device to query.")


No DeckLink device to query.


In [15]:
# Audio Capture Settings

def list_decklink_audio_caps():
    # DeviceMonitor for audio sources
    device_monitor = Gst.DeviceMonitor()
    device_monitor.add_filter("Audio/Source", None)
    device_monitor.start()
    devices = device_monitor.get_devices()
    for dev in devices:
        if 'decklink' in dev.get_display_name().lower():
            print(f"\nAudio Capabilities for: {dev.get_display_name()}")
            caps = dev.get_caps()
            for i in range(caps.get_size()):
                structure = caps.get_structure(i)
                print(f"  {structure.to_string()}")
    device_monitor.stop()

list_decklink_audio_caps()

### GStreamer DeckLink Duo Pipelines

#### VideoSource (1080i5994) - GStreamer Basic Pipeline

gst-launch-1.0 decklinkvideosrc device-number=1 connection=sdi mode=1080i5994 ! decklinkvideosink device-number=0 mode=1080i5994 decklinkaudiosrc device-number=1 ! decklinkaudiosink device-number=0

#### VideoSource (1080p5994) - GStreamer Basic Pipeline

gst-launch-1.0 decklinkvideosrc device-number=1 connection=sdi mode=1080p5994 ! decklinkvideosink device-number=0 mode=1080p5994 decklinkaudiosrc device-number=1 ! decklinkaudiosink device-number=0

#### Input[1]: 1080p59.94 -> 1-sec buffer -> CPU(Proccessing) -> 1-sec buffer -> Output: 1080p29.97 - BGRA

gst-launch-1.0 decklinkvideosrc device-number=1 mode=1080p5994 drop-no-signal-frames=true ! videorate ! queue leaky=downstream max-size-buffers=0 max-size-bytes=0 max-size-time=1000000000 ! videoconvert ! video/x-raw,format=BGRA,width=1920,height=1080,interlace-mode=progressive,framerate=30000/1001 ! queue leaky=downstream max-size-buffers=0 max-size-bytes=0 max-size-time=1000000000 ! decklinkvideosink device-number=0 mode=1080p2997 sync=false video-format=8bit-bgra decklinkaudiosrc device-number=1 ! queue leaky=downstream max-size-buffers=0 max-size-bytes=0 max-size-time=1000000000 ! queue leaky=downstream max-size-buffers=0 max-size-bytes=0 max-size-time=1000000000 ! decklinkaudiosink device-number=0 sync=false

#### GPU CUDA Input[1]: 1080p59.94 -> 1-sec buffer -> GPU(CUDA) -> 1-sec buffer -> Output: 1080p29.97 - BGRA

- https://gstreamer.freedesktop.org/documentation/nvcodec/index.html?gi-language=python

gst-launch-1.0 decklinkvideosrc device-number=1 connection=sdi mode=1080p5994 ! queue max-size-time=1000000000 name=input_buffer ! videoconvert ! video/x-raw,format=BGRA,width=1920,height=1080 ! cudaupload ! cudaconvert ! video/x-raw\(memory:CUDAMemory\),format=BGRA ! identity name=gpu_processing ! cudadownload ! queue max-size-time=1000000000 name=output_buffer ! videorate ! video/x-raw,format=BGRA,width=1920,height=1080,framerate=30000/1001 ! videoconvert ! decklinkvideosink device-number=0 mode=1080p2997 decklinkaudiosrc device-number=1 ! queue max-size-time=1000000000 ! decklinkaudiosink device-number=0

GST_DEBUG_DUMP_DOT_DIR=./ gst-launch-1.0 decklinkvideosrc device-number=1 connection=sdi mode=1080p5994 ! queue max-size-time=1000000000 name=input_buffer ! videoconvert ! video/x-raw,format=BGRA,width=1920,height=1080 ! cudaupload ! cudaconvert ! video/x-raw\(memory:CUDAMemory\),format=BGRA ! identity name=gpu_processing ! cudadownload ! queue max-size-time=1000000000 name=output_buffer ! videorate ! video/x-raw,format=BGRA,width=1920,height=1080,framerate=30000/1001 ! videoconvert ! decklinkvideosink device-number=0 mode=1080p2997 decklinkaudiosrc device-number=1 ! queue max-size-time=1000000000 ! decklinkaudiosink device-number=0

dot -Tsvg 0.00.54.920377760-gst-launch.PLAYING_PAUSED.dot -o 05_in-gpu-out.svg